# Chapter3: Effective Functions

Todos:
- [x] 3.1 Python's Functions Are First-Class
- [x] 3.2 Lambdas Are Single-Expression Functions
- [x] 3.3 The Power of Decorators
- [x] 3.4 Fun With `*args` and `**kwargs`
- [x] 3.5 Function Argument Unpacking
- [ ] 3.6 Nothing to Return Here

## 3.1 Python's Functions Are First-Class

“Python’s functions are first-class objects. You can assign them to variables, store them in data structures, pass them as arguments to other functions, and even return them as values from other functions.

Grokking these concepts intuitively will make understanding advanced features in Python like lambdas and decorators much easier. It also puts you on a path towards functional programming techniques.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [1]:
def yell(text):
    return text.upper() + '!'

In [2]:
yell('hello')

'HELLO!'

### Functions Are Objects

“All data in a Python program is represented by objects or relations between objects. Things like strings, lists, modules, and functions are all objects. There’s nothing particularly special about functions in Python. They’re also just objects.

A variable pointing to a function and the function itself are really two separate concerns.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [3]:
bark = yell

In [4]:
bark('woof')

'WOOF!'

In [5]:
del yell

In [6]:
yell('hello?')

NameError: name 'yell' is not defined

In [7]:
bark('hey')

'HEY!'

In [8]:
bark.__name__

'yell'

### Functions Can Be Stored in Data Structures

In [10]:
funcs = [bark, str.lower, str.capitalize]
funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [11]:
for f in funcs:
    print(f, f('hey there'))

<function yell at 0x7fcce09d9510> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there


In [12]:
funcs[0]('heyho')

'HEYHO!'

### Functions Can Be Passed to Other Functions

In [13]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

### Functions Can Be Nested

In [14]:
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)

In [15]:
speak('Hello, World')

'hello, world...'

In [16]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper

In [17]:
get_speak_func(0.3)

<function __main__.get_speak_func.<locals>.whisper(text)>

In [18]:
get_speak_func(0.7)

<function __main__.get_speak_func.<locals>.yell(text)>

In [19]:
get_speak_func(0.7)('Hello')

'HELLO!'

### Functions Can Capture Local State

“A closure remembers the values from its enclosing lexical scope even when the program flow is no longer in that scope.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [20]:
def make_adder(n):
    def add(x):
        return x + n
    return add

In [21]:
plus_3 = make_adder(3)

In [22]:
plus_3(4)

7

In [23]:
def make_adder2(n):
    a = 3
    def add(x):
        return x + a + n
    return add

In [27]:
plus_6 = make_adder2(3)

In [28]:
plus_6(4)

10

### Objects Can Behave Like Functions

“While all functions are objects in Python, the reverse isn’t true. Objects aren’t functions. But they can be made callable, which allows you to treat them like functions in many cases.

If an object is callable it means you can use the round parentheses function call syntax on it and even pass in function call arguments. This is all powered by the __call__ dunder method.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [29]:
class Adder:
    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        return self.n + x

In [31]:
plus_3 = Adder(3)
plus_3(4)

7

In [32]:
callable(plus_3)

True

In [34]:
callable('hello')

False

### Key Takeaways

- Everything in Python is an object, including functions. You can assign them to variables, store them in data structures, and pass or return them to and from other functions (first-class functions.)
- First-class functions allow you to abstract away and pass around behavior in your programs.
- Functions can be nested and they can capture and carry some of the parent function’s state with them. Functions that do this are called closures.
- Objects can be made callable. In many cases this allows you to treat them like functions.

## 3.2 Lambdas Are Single-Expression Functions

In [35]:
add = lambda x, y: x + y
add(5, 3)

8

In [36]:
(lambda x, y: x + y)(5, 3)

8

### But Maybe You Shouldn't...

In [37]:
#Harmful
list(filter(lambda x: x % 2 == 0, range(1, 6)))

[2, 4]

In [38]:
#Better
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

### Key Takeaways

- Lambda functions are single-expression functions that are not necessarily bound to a name (anonymous).
- Lambda functions can’t use regular Python statements and always include an implicit return statement.
- Always ask yourself: Would using a regular (named) function or a list comprehension offer more clarity?

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

## 3.3 The Power of Decorators

“At their core, Python’s decorators allow you to extend and modify the behavior of a callable (functions, methods, and classes) without permanently modifying the callable itself.

Any sufficiently generic functionality you can tack on to an existing class or function’s behavior makes a great use case for decoration. This includes the following:

- logging
- enforcing access control and authentication
- instrumentation and timing functions
- rate-limiting
- caching, and more”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

### Python Decorator Basics

“Now, what are decorators really? They “decorate” or “wrap” another function and let you execute code before and after the wrapped function runs.

Decorators allow you to define reusable building blocks that can change or extend the behavior of other functions. And, they let you do that without permanently modifying the wrapped function itself. The function’s behavior changes only when it’s decorated.

What might the implementation of a simple decorator look like? In basic terms, a decorator is a callable that takes a callable as input and returns another callable.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [1]:
def null_decorator(func):
    return func

In [3]:
def greet():
    return 'Hello!'

In [4]:
greet = null_decorator(greet)
greet()

'Hello!'

In [5]:
@null_decorator
def greet():
    return 'Hello!'

greet()

'Hello!'

### Decorators Can Modify Behavior

In [6]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

In [7]:
@uppercase
def greet():
    return 'Hello!'

greet()

'HELLO!'

### Applying Multiple Decorators to a Function

In [11]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

In [12]:
@strong
@emphasis
def greet():
    return 'Hello!'

greet()

'<strong><em>Hello!</em></strong>'

In [16]:
def greet():
    return 'Hello!'

decorated_greet = strong(emphasis(greet))
decorated_greet()

'<strong><em>Hello!</em></strong>'

### Decorating Functions That Accept Arguments

- It uses the * and ** operators in the wrapper closure definition to collect all positional and keyword arguments and stores them in variables (args and kwargs).
- The wrapper closure then forwards the collected arguments to the original input function using the * and ** “argument unpacking” operators.

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [17]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

In [35]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() 'f'with {args}, {kwargs}')
        original_result = func(*args, **kwargs)
        print(f'TRACE: {func.__name__}() 'f'returned {original_result!r}')
        return original_result
    return wrapper

In [36]:
@trace
def say(name, line):
    return f'{name}: {line}'

say('Jane', 'Hello, World')

TRACE: calling say() with ('Jane', 'Hello, World'), {}
TRACE: say() returned 'Jane: Hello, World'


'Jane: Hello, World'

### How to Write "Debuggable" Decorators

“For example, the original function name, its docstring, and parameter list are hidden by the wrapper closure”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [37]:
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

decorated_greet = uppercase(greet)

In [39]:
greet.__name__

'greet'

In [40]:
greet.__doc__

'Return a friendly greeting.'

In [41]:
decorated_greet.__name__

'wrapper'

In [42]:
decorated_greet.__doc__

“You can use functools.wraps in your own decorators to copy over the lost metadata from the undecorated function to the decorator closure.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [62]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

In [63]:
@uppercase
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

In [64]:
greet.__name__

'greet'

In [65]:
greet.__doc__

'Return a friendly greeting.'

In [66]:
greet()

'HELLO!'

### Key Takeaways

- Decorators define reusable building blocks you can apply to a callable to modify its behavior without permanently modifying the callable itself.
- The @ syntax is just a shorthand for calling the decorator on an input function. Multiple decorators on a single function are applied bottom to top (decorator stacking).
- As a debugging best practice, use the functools.wraps helper in your own decorators to carry over metadata from the undecorated callable to the decorated one.”
- Just like any other tool in the software development toolbox, decorators are not a cure-all and they should not be overused. It’s important to balance the need to “get stuff done” with the goal of “not getting tangled up in a horrible, unmaintainable mess of a code base.

## 3.4 Fun With `*args` and `**kwargs`

“I want to make it clear that calling the parameters args and kwargs is simply a naming convention. The previous example would work just as well if you called them `*parms` and `**argv`. The actual syntax is just the asterisk (*) or double asterisk (**), respectively.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [68]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

In [69]:
foo()

TypeError: foo() missing 1 required positional argument: 'required'

In [70]:
foo('hello')

hello


In [71]:
foo('hello', 1, 2, 3)

hello
(1, 2, 3)


In [72]:
foo('hello', 1, 2, 3, key1='value', key2=999)

hello
(1, 2, 3)
{'key1': 'value', 'key2': 999}


### Forwarding Optional or Keyword Arguments

“It’s possible to pass optional or keyword parameters from one function to another. You can do so by using the argument-unpacking operators * and ** when calling the function you want to forward arguments to.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [73]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', )
    bar(x, *new_args, **kwargs)

“This technique can be useful for subclassing and writing wrapper functions. For example, you can use it to extend the behavior of a parent class without having to replicate the full signature of its constructor in the child class. This can be quite convenient if you’re working with an API that might change outside of your control.

Typically you wouldn’t use this technique with your own class hierarchies. The more likely scenario would be that you’ll want to modify or override behavior in some external class which you don’t control.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [74]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

class AlwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.color = 'blue'

AlwaysBlueCar('green', 48392).color

'blue'

In [75]:
def trace(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        print(f, args, kwargs)
        result = f(*args, **kwargs)
        print(result)
    return decorated_function

@trace
def greet(greeting, name):
    return f'{greeting}, {name}!'

greet('Hello', 'Bob')

<function greet at 0x7fc5b8ff17b8> ('Hello', 'Bob') {}
Hello, Bob!


### Key Takeaways

- `*args` and `**kwargs` let you write functions with a variable number of arguments in Python.
- `*args` collects extra positional arguments as a tuple. `**kwargs` collects the extra keyword arguments as a dictionary.
- The actual syntax is * and **. Calling them args and kwargs is just a convention (and one you should stick to).”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

## 3.5 Function Argument Unpacking

“This technique works for any iterable, including generator expressions. Using the * operator on a generator consumes all elements from the generator and passes them to the function”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [76]:
def print_vector(x, y, z):
    print(f'<{x}, {y}, {z}>')

In [77]:
print_vector(0, 1, 0)

<0, 1, 0>


In [79]:
tuple_vec = (1, 0, 1)
print_vector(tuple_vec[0], tuple_vec[1], tuple_vec[2])

<1, 0, 1>


In [80]:
list_vec = [1, 0, 1]
print_vector(list_vec[0], list_vec[1], list_vec[2])

<1, 0, 1>


In [81]:
print_vector(*tuple_vec)

<1, 0, 1>


In [82]:
print_vector(*list_vec)

<1, 0, 1>


In [83]:
genexpr = (x * x for x in range(3))
print_vector(*genexpr)

<0, 1, 4>


“Besides the * operator for unpacking sequences like tuples, lists, and generators into positional arguments, there’s also the ** operator for unpacking keyword arguments from dictionaries.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [84]:
dict_vec = {'y': 0, 'z': 1, 'x': 1}
print_vector(**dict_vec)

<1, 0, 1>


“If you were to use the single asterisk (*) operator to unpack the dictionary, keys would be passed to the function in random order instead”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [86]:
print_vector(*dict_vec)

<y, z, x>


### Key Takeaways

- The * and ** operators can be used to “unpack” function arguments from sequences and dictionaries.
- Using argument unpacking effectively can help you write more flexible interfaces for your modules and functions.

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 